In [38]:
# import os
# os.environ['REQUESTS_CA_BUNDLE'] = r'C:\One-Drive\OneDrive - Tredence\Documents\LLM Engineer Track Program\Capstone Project\Zscaler Root CA.crt'

In [79]:
from langchain_community.document_loaders import WikipediaLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain

In [80]:
import pandas as pd
import os
from datetime import datetime
from dotted_dict import DottedDict
from langchain_community.vectorstores import Chroma
from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings

In [81]:
azure_config = {
    "model_deployment": "GPT_35_TURBO_DEMO_RAG_DEPLOYMENT_DONO",
    "embedding_deployment": "ADA_RAG_DONO_DEMO",
    "embedding_name": "text-embedding-ada-002",
    "api_key":"f6f4b8aec16b4094bfe0b8e063dbf1a3",
    "api_version":"2024-02-01",
    "endpoint":"https://dono-rag-demo-resource-instance.openai.azure.com/",
    "model":"GPT_35_TURBO_DEMO_RAG_DEPLOYMENT_DONO"
    }

In [82]:
def models(azure_config):
    print("developing models")
    models=DottedDict()
    llm = AzureChatOpenAI(temperature=0,
                      api_key=azure_config["api_key"],
                      openai_api_version=azure_config["api_version"],
                      azure_endpoint=azure_config["endpoint"],
                      model=azure_config["model_deployment"],
                      validate_base_url=False)
    embedding_model = AzureOpenAIEmbeddings(
        api_key=azure_config["api_key"],
        openai_api_version=azure_config["api_version"],
        azure_endpoint=azure_config["endpoint"],
        model = azure_config["embedding_deployment"]
    )
    models.llm=llm
    models.embedding_model=embedding_model 
    return models


In [83]:
models=models(azure_config)

developing models


In [84]:
def load_wikipedia(query):
    from langchain_community.document_loaders import WikipediaLoader
    loader = WebBaseLoader(web_path=query)
    data = loader.load()
    return data

In [85]:
doc=load_wikipedia('https://en.wikipedia.org/wiki/FIFA_World_Cup')

In [86]:
type(doc)

list

In [87]:
def chunking(data):
    from langchain_text_splitters import RecursiveCharacterTextSplitter
    text_splitter =RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=30, length_function=len, is_separator_regex=False, separators=["."])
    chunk = text_splitter.split_documents(data)
    return chunk

In [88]:
chunked_data=chunking(doc)

In [89]:
def get_vector_store(text_chunks, models):
    embeddings = models.embedding_model
    vectordb=FAISS.from_documents(chunked_data,embeddings)
    return vectordb

In [90]:
def get_conversational_chain(llm):
    prompt_template = """
    Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in
    provided context just say, "answer is not available in the context", don't provide the wrong answer\n\n
    Context:\n {context}?\n
    Question: \n{question}\n

    Answer:
    """
    prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
    chain = load_qa_chain(llm, chain_type="stuff", prompt=prompt)
    return chain


In [91]:
def get_answer(query):
    document_search=get_vector_store(chunked, models)
    similar_docs = document_search.similarity_search(query, k=1) # get closest chunks
    chain = get_conversational_chain(models.llm)
    answer = chain.invoke(input={"input_documents": similar_docs, "question": query}, return_only_outputs=True)
    return answer

In [92]:
answer = get_answer('Who was the host of 1982 fifa world cup')

In [93]:
answer

{'output_text': 'Spain was the host of the 1982 FIFA World Cup.'}

In [94]:
from langchain.memory import ChatMessageHistory
history = ChatMessageHistory()